# Time series forecasting using Radial basis function network ( regression)

Data structure: 
- ID: Product Identifier
- DATEOP: Date Operation (weekly)
- NBRE: Sales/Demands count/quantity

Goal n°1: one-month-ahead forecasting of each product

Goal n°2: 4-month-ahead forecasting of each product.

"Product" refers to an anonymous banking product.


In [1]:
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from pandas import DataFrame
from pandas import concat
from matplotlib.pylab import rcParams
from collections import defaultdict
from scipy.spatial import distance
from sklearn.cluster import KMeans
import numpy as np
from sklearn.neural_network import MLPRegressor
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.optimizers import SGD
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import optimizers 


Using TensorFlow backend.


In [3]:
path=r'C:/Users/yesmi/Desktop/engineerinternship/btseries.txt'
data= pd.read_csv(path,sep=';')

In [4]:
data.head()

,ID,DATEOP,NBRE
0,9243015,30/11/2009,1
1,9243015,31/12/2009,0
2,9243015,31/01/2010,1
3,9243015,28/02/2010,0
4,9243015,31/03/2010,2


In [5]:
data.shape

(3346, 3)

In [6]:
data.dtypes

ID        object
DATEOP    object
NBRE       int64
dtype: object

In [7]:
# 63 products
data.ID.value_counts()

BX7          147
9.33E+15     147
ZZ103160     145
ZZ100767      95
ZZ107182      68
ZZ101783      68
ZZ100639      68
ZZ101831      68
9273121       67
ZZ100121      67
ZZ100202      66
ZZ102716      66
ZZ101787      66
ZZ102536      66
ZZ102485      66
ZZ1050331     66
ZZ102253      66
BX10          66
ZZ102130      66
ZZ103110      65
BX5           65
9243015       64
ZZ100761      64
ZZ101365      63
ZZ103604      63
BG10          63
ZZ1050235     62
BX15          62
ZZ102254      62
ZZ107452      62
            ... 
9283022       56
BG7           55
CBM60         54
ZZF1017       52
ZZ300758      52
9283021       52
9273011       51
BG5           47
ZZ100594      46
ZZ508539      46
9.32E+15      44
BX20          42
ZZ103950      41
ZZ300568      40
ZZ501250      38
ZZ8950        35
BS30          33
ZZ101824      30
ZZ100211      29
ZZ300676      28
ZZ100780      22
9273052       16
ZZ305027      11
9273051        6
9273031        5
BX 25          3
ZZ502209       3
ZZ502052      

In [8]:
data.ID=data.ID.astype('category')

In [9]:
from dateutil import parser
for i in range (data.shape[0]):
    data.DATEOP[i]=parser.parse(data.DATEOP[i])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
#184 dates
data.DATEOP.value_counts()

2010-10-31    56
2010-11-30    56
2010-12-31    55
2011-05-31    54
2010-09-30    53
2011-03-31    53
2011-06-30    53
2011-09-30    53
2011-01-31    53
2011-07-31    53
2011-10-31    53
2011-02-28    53
2011-04-30    53
2011-08-31    53
2012-02-29    52
2011-12-31    52
2011-11-30    52
2012-01-31    51
2012-08-31    51
2010-08-31    51
2012-12-31    51
2012-07-31    51
2010-07-31    51
2012-03-31    51
2012-06-30    51
2012-05-31    51
2012-11-30    51
2012-04-30    51
2010-06-30    50
2013-01-31    50
              ..
2007-09-30     4
2009-07-31     4
2005-04-30     4
2003-05-31     4
2006-04-30     4
2007-06-30     4
2008-07-31     4
2007-10-31     4
2006-02-28     4
2005-08-31     4
2009-06-30     4
2007-07-31     4
2007-08-31     4
2006-06-30     4
2006-03-31     4
2008-08-31     4
2006-05-31     4
2005-06-30     4
2004-11-30     4
2003-02-28     4
2009-04-30     4
2003-06-30     4
2003-12-31     4
2003-03-31     4
2005-12-31     4
2005-02-28     4
2007-05-31     4
2008-05-31    

In [11]:
data.dtypes

ID        category
DATEOP      object
NBRE         int64
dtype: object

# Convert a Time Series forecasting  to a Supervised Learning task

We have a multivariate time series data.
We are going to use one stop sliding window approach to predict one-month-ahead of each product.

In [12]:
#First transformation
dat=data.pivot_table(index='DATEOP',
                             columns='ID',
                             values='NBRE',
                             aggfunc='sum')

In [13]:
dat=pd.DataFrame(dat)
dat.shape

(148, 63)

In [14]:
dat

ID,9.32E+15,9.33E+15,9243015,9273011,9273021,9273031,9273051,9273052,9273121,9283021,...,ZZ300676,ZZ300758,ZZ305027,ZZ501250,ZZ502052,ZZ502209,ZZ508539,ZZ8950,ZZF1013,ZZF1017
DATEOP,,,,,,,,,,,,,,,,,,,,,
2003-01-31,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-02-28,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-03-31,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-04-30,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-05-31,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-06-30,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-07-31,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-08-31,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2003-09-30,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
Products=dat.columns
Products

CategoricalIndex(['9.32E+15', '9.33E+15', '9243015', '9273011', '9273021',
                  '9273031', '9273051', '9273052', '9273121', '9283021',
                  '9283022', 'BG10', 'BG5', 'BG7', 'BS30', 'BX 25', 'BX10',
                  'BX15', 'BX20', 'BX5', 'BX7', 'CBM', 'CBM60', 'ZZ100121',
                  'ZZ100202', 'ZZ100211', 'ZZ100594', 'ZZ100639', 'ZZ100761',
                  'ZZ100767', 'ZZ100780', 'ZZ101365', 'ZZ101783', 'ZZ101787',
                  'ZZ101824', 'ZZ101831', 'ZZ102130', 'ZZ102253', 'ZZ102254',
                  'ZZ102485', 'ZZ102536', 'ZZ102716', 'ZZ103110', 'ZZ103160',
                  'ZZ103604', 'ZZ103950', 'ZZ105007', 'ZZ1050235', 'ZZ1050331',
                  'ZZ107182', 'ZZ107208', 'ZZ107452', 'ZZ300568', 'ZZ300676',
                  'ZZ300758', 'ZZ305027', 'ZZ501250', 'ZZ502052', 'ZZ502209',
                  'ZZ508539', 'ZZ8950', 'ZZF1013', 'ZZF1017'],
                 categories=['9.32E+15', '9.33E+15', '9243015', '9273011', '9273021', '92

In [16]:
dat=dat.fillna(value=0)

In [17]:
#second transformation

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 
 


In [18]:
import numpy as np
dat=np.array(dat)
dat

array([[  0.,   1.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ..., 
       [  0.,  35.,   3., ...,   0.,   0.,   0.],
       [  0.,  31.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [19]:
#we will be using the 10 final dates ( for each product) to predict the value of the next month
dt=series_to_supervised(dat,10,1)

In [20]:
dt.shape

(138, 693)

In [21]:
dt.head(20)

,var1(t-10),var2(t-10),var3(t-10),var4(t-10),var5(t-10),var6(t-10),var7(t-10),var8(t-10),var9(t-10),var10(t-10),...,var54(t),var55(t),var56(t),var57(t),var58(t),var59(t),var60(t),var61(t),var62(t),var63(t)
10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Applying RBFN to predict the value of the first product

An example to validate the model

In [22]:
X_train, X_test, y_train, y_test = train_test_split( dt.loc[:,'var1(t-10)':'var63(t-1)'] , dt.loc[:,'var1(t)':'var63(t)'] , test_size=0.30, random_state=42)

In [23]:
y_train.shape

(96, 63)

In [24]:
y_test.shape

(42, 63)

In [25]:
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

In [26]:
y_train.columns=Products
y_test.columns=Products

In [27]:
# Gaussian function

def RBFunction(r, x,c ):
    return( np.exp(- (distance.euclidean(x, c))**2/r**2))

# Computing radial/spread

def radial (x,c ):
    r=[]
    for i in range (c.shape[0]):
        k=[]
        for j in range(x.shape[0]) :
            k.append(distance.euclidean(x.iloc[j,:], c[i])) 
        m=(sum(k)/len(k))
        r.append(m)
    return(r)

# compute the centers using Kmeans

def centers (data,n):
    kmeans = KMeans(n_clusters=n)
    a = kmeans.fit(data)
    return(kmeans.cluster_centers_)     

#Preparing the RBF network

def RBF (centers,radials, X): 
    s=[[]for k in range (centers.shape[0])]
    
    for i in range (centers.shape[0]):
        for j in range (X.shape[0]):
            X=pd.DataFrame(X)
            s[i] .append(RBFunction(radials[i] ,X.iloc[j,:],centers[i]))
    s=pd.DataFrame(s)
    return(s.transpose()) 


In [28]:
# function to evaluate the performance of the models
def evaluate(y,pred):
    #MAPE:Mean absolute percentage error
    def MAPE(y,pred):
        mapev=[]
        aux1=y.reset_index(drop=True)
        aux1=pd.Series(aux1)
        aux2=pd.Series(pred)

        for i in aux1.index:
            if aux1[i]==0:
                mapev.append(0)
            else:
                mapev.append(abs(aux1[i]-aux2[i])/aux1[i])

        mape=(np.mean(mapev))*100
        return(mape)

#MAD:Mean absolute deviation
    def MAD_MSD(y,pred):
        madv=[]
        mapev=[]
        aux1=y.reset_index(drop=True)
        aux1=pd.Series(aux1)
        aux2=pd.Series(pred)
        for i in aux1.index:
            if aux1[i]==0:
                mapev.append(0)
            else:
                mapev.append(abs(aux1[i]-aux2[i])/aux1[i])
        for i in aux1.index:
            if aux1[i]!=0:
               madv.append(mapev[i]*aux1[i])
            else:
               madv.append(0)
        mad=np.mean(madv)
        #MSD:Mean squared deviation
        msdv=np.square(madv)
        msd=np.mean(msdv)
        return(mad,msd)

#SMAPE:Symmetric mean absolute percentage error
    def SMAPE(y,pred):
       smapev=[]
       smape=0
       aux1=y.reset_index(drop=True)
       aux1=pd.Series(aux1)
       aux2=pd.Series(pred)
       for i in aux1.index:
           if aux1[i]==0:
              smapev.append(0)
           else:
               smapev.append(2*(abs(aux1[i]-aux2[i])/((abs(aux1[i])+abs(aux2[i])))))

       smape = (np.mean(smapev))*100
       return(smape)
    MAPE=MAPE(y,pred)
    MAD,MSD=MAD_MSD(y,pred)
    SMAPE=SMAPE(y,pred)
    nn=['MAPE','MAD','MSD','SMAPE']
    kk=[MAPE,MAD,MSD,SMAPE]
    eva=pd.DataFrame([nn,kk])
    return(eva)
    

In [29]:
#neural network initialization
# Hyperparameter: number of centers/units of the hidden layer nc
nc=80
#Train
c=centers(X_train , nc)
r=radial(X_train,c)
inputs_train= pd.DataFrame(RBF(c,r,X_train))

#Test
inputs_t= pd.DataFrame(RBF(c,r,X_test))

## Starting with linear regression

In [30]:
#linear model
from sklearn import linear_model
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
mr=regr.fit(X_train, y_train.iloc[:,0])

# Make predictions using the testing set
y_pred = regr.predict(X_test)

In [31]:
#Test
var=explained_variance_score(y_test.iloc[:,0], y_pred) 
mae=mean_absolute_error(y_test.iloc[:,0], y_pred)
mse=mean_squared_error(y_test.iloc[:,0], y_pred)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], y_pred )  
metric=[var,mae,mse,rmse,r2]
nn=['Explained variance','MAE','MSE','RMSE','R^2']
performance=pd.DataFrame([nn,metric])
performance

,0,1,2,3,4
0,Explained variance,MAE,MSE,RMSE,R^2
1,-4.79943e+21,1.57526e+11,9.47156e+22,3.07759e+11,-4.83878e+21


In [32]:
eval1=evaluate( y_test.iloc[:,0],y_pred)
eval1

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,9.4226e+11,5.74291e+10,3.7741e+22,38.0952


## Using Sklearn package (MLPRegressor) to implement the second part of the RBFNetwork

In [33]:
m=MLPRegressor(activation='identity',alpha=0.05,learning_rate ='adaptive',verbose=False)

In [34]:
mod=m.fit(inputs_train,y_train.iloc[:,0])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [35]:
#Train
out=m.predict(inputs_train)
var=explained_variance_score(y_train.iloc[:,0], out) 
mae=mean_absolute_error(y_train.iloc[:,0], out)
mse=mean_squared_error(y_train.iloc[:,0], out)
rmse=np.sqrt(mse)
r2=r2_score(y_train.iloc[:,0], out ) 
#r2a=1-((1-r2)* (1000-1) /(1000-1-5)) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

,0,1,2,3,4
0,Explained variance,MAE,MSE,RMSE,R^2
1,0.748067,1.54814,6.55893,2.56104,0.747471


In [36]:
evaluate(y_train.iloc[:,0], out)

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,20.1189,0.896115,4.82718,15.571


In [37]:
#Test
outt=m.predict(inputs_t)
var=explained_variance_score(y_test.iloc[:,0], outt) 
mae=mean_absolute_error(y_test.iloc[:,0], outt)
mse=mean_squared_error(y_test.iloc[:,0], outt)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], outt) 
#r2a=1-((1-r2)* (1000-1) /(1000-1-5)) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

,0,1,2,3,4
0,Explained variance,MAE,MSE,RMSE,R^2
1,0.5009,1.62935,9.77302,3.12618,0.500721


In [38]:
evaluate(y_test.iloc[:,0], outt)

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,14.4298,0.995049,7.89957,13.4683


## Using Kears package to implement the second part of the RBFNetwork

In [39]:
#def baseline_model():
    # create model
model = Sequential()
model.add(Dense(1, input_dim=nc, kernel_initializer='normal'))
    # Compile model
model.compile(loss='mean_squared_error', optimizer='sgd')

#data preparation
inputs_train_arr=np.array(inputs_train)
inputs_t_arr=np.array(inputs_t)
y_train_arr=np.array(y_train.iloc[:,0])
y_t_arr=np.array(y_test.iloc[:,0])

#train the model
model.fit(inputs_train_arr,y_train_arr, epochs=3000, batch_size=20,verbose=1)

Epoch 1/3000
96/96 [==============================] - 0s - loss: 29.7128     
Epoch 2/3000
96/96 [==============================] - 0s - loss: 26.9773     
Epoch 3/3000
96/96 [==============================] - 0s - loss: 25.6713     
Epoch 4/3000
96/96 [==============================] - 0s - loss: 25.0835     
Epoch 5/3000
96/96 [==============================] - 0s - loss: 24.9362    
Epoch 6/3000
96/96 [==============================] - 0s - loss: 24.3246     
Epoch 7/3000
96/96 [==============================] - 0s - loss: 24.3335     
Epoch 8/3000
96/96 [==============================] - 0s - loss: 24.0210     
Epoch 9/3000
96/96 [==============================] - 0s - loss: 23.9025     
Epoch 10/3000
96/96 [==============================] - 0s - loss: 23.6098     
Epoch 11/3000
96/96 [==============================] - 0s - loss: 23.3858     
Epoch 12/3000
96/96 [==============================] - 0s - loss: 23.2398    
Epoch 13/3000
96/96 [==============================] - 0s - los

96/96 [==============================] - 0s - loss: 14.1159    
Epoch 105/3000
96/96 [==============================] - 0s - loss: 13.7770     
Epoch 106/3000
96/96 [==============================] - 0s - loss: 13.7084     
Epoch 107/3000
96/96 [==============================] - 0s - loss: 13.6073     
Epoch 108/3000
96/96 [==============================] - 0s - loss: 13.5822    
Epoch 109/3000
96/96 [==============================] - 0s - loss: 13.5277     
Epoch 110/3000
96/96 [==============================] - 0s - loss: 13.4111     
Epoch 111/3000
96/96 [==============================] - 0s - loss: 13.3603    
Epoch 112/3000
96/96 [==============================] - 0s - loss: 13.3050    
Epoch 113/3000
96/96 [==============================] - 0s - loss: 13.3051    
Epoch 114/3000
96/96 [==============================] - 0s - loss: 13.3598     
Epoch 115/3000
96/96 [==============================] - 0s - loss: 13.2933     
Epoch 116/3000
96/96 [==============================] - 0s -

96/96 [==============================] - 0s - loss: 9.2883     
Epoch 208/3000
96/96 [==============================] - 0s - loss: 9.3574     
Epoch 209/3000
96/96 [==============================] - 0s - loss: 9.2637     
Epoch 210/3000
96/96 [==============================] - 0s - loss: 9.2468     
Epoch 211/3000
96/96 [==============================] - 0s - loss: 9.2575      
Epoch 212/3000
96/96 [==============================] - 0s - loss: 9.1537     
Epoch 213/3000
96/96 [==============================] - 0s - loss: 9.2232      
Epoch 214/3000
96/96 [==============================] - 0s - loss: 9.1245     
Epoch 215/3000
96/96 [==============================] - 0s - loss: 9.0641      
Epoch 216/3000
96/96 [==============================] - 0s - loss: 9.0408      
Epoch 217/3000
96/96 [==============================] - 0s - loss: 9.0624      
Epoch 218/3000
96/96 [==============================] - 0s - loss: 8.9581     
Epoch 219/3000
96/96 [==============================] - 0s - l

96/96 [==============================] - 0s - loss: 7.1014      
Epoch 310/3000
96/96 [==============================] - 0s - loss: 7.0292     
Epoch 311/3000
96/96 [==============================] - 0s - loss: 6.9684      
Epoch 312/3000
96/96 [==============================] - 0s - loss: 6.9926      
Epoch 313/3000
96/96 [==============================] - 0s - loss: 7.0185     
Epoch 314/3000
96/96 [==============================] - 0s - loss: 6.9504     
Epoch 315/3000
96/96 [==============================] - 0s - loss: 6.9289     
Epoch 316/3000
96/96 [==============================] - 0s - loss: 6.9654      
Epoch 317/3000
96/96 [==============================] - 0s - loss: 6.9052     
Epoch 318/3000
96/96 [==============================] - 0s - loss: 6.8652     
Epoch 319/3000
96/96 [==============================] - 0s - loss: 6.8619      
Epoch 320/3000
96/96 [==============================] - 0s - loss: 6.8237      
Epoch 321/3000
96/96 [==============================] - 0s - 

96/96 [==============================] - 0s - loss: 5.7551     
Epoch 412/3000
96/96 [==============================] - 0s - loss: 5.7110     
Epoch 413/3000
96/96 [==============================] - 0s - loss: 5.7161     
Epoch 414/3000
96/96 [==============================] - 0s - loss: 5.7209     
Epoch 415/3000
96/96 [==============================] - 0s - loss: 5.6892     
Epoch 416/3000
96/96 [==============================] - 0s - loss: 5.6466      
Epoch 417/3000
96/96 [==============================] - 0s - loss: 5.6702     
Epoch 418/3000
96/96 [==============================] - 0s - loss: 5.6393     
Epoch 419/3000
96/96 [==============================] - 0s - loss: 5.6232     
Epoch 420/3000
96/96 [==============================] - 0s - loss: 5.6342     
Epoch 421/3000
96/96 [==============================] - 0s - loss: 5.6035     
Epoch 422/3000
96/96 [==============================] - 0s - loss: 5.6222     
Epoch 423/3000
96/96 [==============================] - 0s - loss:

...

96/96 [==============================] - 0s - loss: 1.6909     
Epoch 2547/3000
96/96 [==============================] - 0s - loss: 1.7224     
Epoch 2548/3000
96/96 [==============================] - 0s - loss: 1.7390     
Epoch 2549/3000
96/96 [==============================] - 0s - loss: 1.7179     
Epoch 2550/3000
96/96 [==============================] - 0s - loss: 1.7127     
Epoch 2551/3000
96/96 [==============================] - 0s - loss: 1.6943     
Epoch 2552/3000
96/96 [==============================] - 0s - loss: 1.7017     
Epoch 2553/3000
96/96 [==============================] - 0s - loss: 1.7065     
Epoch 2554/3000
96/96 [==============================] - 0s - loss: 1.6833     
Epoch 2555/3000
96/96 [==============================] - 0s - loss: 1.7073     
Epoch 2556/3000
96/96 [==============================] - 0s - loss: 1.7080     
Epoch 2557/3000
96/96 [==============================] - 0s - loss: 1.6973     
Epoch 2558/3000
96/96 [==============================] -

96/96 [==============================] - 0s - loss: 1.6488     
Epoch 2649/3000
96/96 [==============================] - 0s - loss: 1.6822     
Epoch 2650/3000
96/96 [==============================] - 0s - loss: 1.6422     
Epoch 2651/3000
96/96 [==============================] - 0s - loss: 1.6303     
Epoch 2652/3000
96/96 [==============================] - 0s - loss: 1.6430     
Epoch 2653/3000
96/96 [==============================] - 0s - loss: 1.6506     
Epoch 2654/3000
96/96 [==============================] - 0s - loss: 1.6354     
Epoch 2655/3000
96/96 [==============================] - 0s - loss: 1.6565     
Epoch 2656/3000
96/96 [==============================] - 0s - loss: 1.6394     
Epoch 2657/3000
96/96 [==============================] - 0s - loss: 1.6536     
Epoch 2658/3000
96/96 [==============================] - 0s - loss: 1.6420     
Epoch 2659/3000
96/96 [==============================] - 0s - loss: 1.6358     
Epoch 2660/3000
96/96 [==============================] -

96/96 [==============================] - 0s - loss: 1.5938     
Epoch 2751/3000
96/96 [==============================] - 0s - loss: 1.5904     
Epoch 2752/3000
96/96 [==============================] - 0s - loss: 1.6002     
Epoch 2753/3000
96/96 [==============================] - ETA: 0s - loss: 1.769 - 0s - loss: 1.5986     
Epoch 2754/3000
96/96 [==============================] - 0s - loss: 1.6198     
Epoch 2755/3000
96/96 [==============================] - 0s - loss: 1.5938     
Epoch 2756/3000
96/96 [==============================] - 0s - loss: 1.6308     
Epoch 2757/3000
96/96 [==============================] - 0s - loss: 1.6142     
Epoch 2758/3000
96/96 [==============================] - 0s - loss: 1.6039     
Epoch 2759/3000
96/96 [==============================] - 0s - loss: 1.6006     
Epoch 2760/3000
96/96 [==============================] - 0s - loss: 1.6047     
Epoch 2761/3000
96/96 [==============================] - 0s - loss: 1.5933     
Epoch 2762/3000
96/96 [=========

96/96 [==============================] - 0s - loss: 1.6083     
Epoch 2852/3000
96/96 [==============================] - 0s - loss: 1.5617     
Epoch 2853/3000
96/96 [==============================] - 0s - loss: 1.5502     
Epoch 2854/3000
96/96 [==============================] - 0s - loss: 1.5493     
Epoch 2855/3000
96/96 [==============================] - 0s - loss: 1.5475     
Epoch 2856/3000
96/96 [==============================] - 0s - loss: 1.5589     
Epoch 2857/3000
96/96 [==============================] - 0s - loss: 1.5515     
Epoch 2858/3000
96/96 [==============================] - 0s - loss: 1.5381     
Epoch 2859/3000
96/96 [==============================] - 0s - loss: 1.5466     
Epoch 2860/3000
96/96 [==============================] - 0s - loss: 1.5440     
Epoch 2861/3000
96/96 [==============================] - 0s - loss: 1.5348     
Epoch 2862/3000
96/96 [==============================] - 0s - loss: 1.5485     
Epoch 2863/3000
96/96 [==============================] -

96/96 [==============================] - 0s - loss: 1.4957     
Epoch 2954/3000
96/96 [==============================] - 0s - loss: 1.5032     
Epoch 2955/3000
96/96 [==============================] - 0s - loss: 1.5111     
Epoch 2956/3000
96/96 [==============================] - 0s - loss: 1.5009     
Epoch 2957/3000
96/96 [==============================] - 0s - loss: 1.4972     
Epoch 2958/3000
96/96 [==============================] - 0s - loss: 1.4926     
Epoch 2959/3000
96/96 [==============================] - 0s - loss: 1.4968     
Epoch 2960/3000
96/96 [==============================] - 0s - loss: 1.5284     
Epoch 2961/3000
96/96 [==============================] - 0s - loss: 1.4976     
Epoch 2962/3000
96/96 [==============================] - 0s - loss: 1.5126     
Epoch 2963/3000
96/96 [==============================] - 0s - loss: 1.5045     
Epoch 2964/3000
96/96 [==============================] - 0s - loss: 1.5023     
Epoch 2965/3000
96/96 [==============================] -

In [40]:
#Train
out=model.predict(inputs_train_arr)
var=explained_variance_score(y_train.iloc[:,0], out) 
mae=mean_absolute_error(y_train.iloc[:,0], out)
mse=mean_squared_error(y_train.iloc[:,0], out)
rmse=np.sqrt(mse)
r2=r2_score(y_train.iloc[:,0], out ) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

,0,1,2,3,4
0,Explained variance,MAE,MSE,RMSE,R^2
1,0.943392,0.679487,1.47976,1.21645,0.943027


In [41]:
out=np.array(out)
out=np.squeeze(out)
evaluate(y_train.iloc[:,0], out)

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,9.63167,0.381761,0.982615,10.272


In [42]:
#Test
outt=model.predict(inputs_t_arr)
var=explained_variance_score(y_test.iloc[:,0], outt) 
mae=mean_absolute_error(y_test.iloc[:,0], outt)
mse=mean_squared_error(y_test.iloc[:,0], outt)
rmse=np.sqrt(mse)
r2=r2_score(y_test.iloc[:,0], outt) 
nn=['Explained variance','MAE','MSE','RMSE','R^2']
metric=[var,mae,mse,rmse,r2]
performance=pd.DataFrame([nn,metric])
performance

,0,1,2,3,4
0,Explained variance,MAE,MSE,RMSE,R^2
1,0.577746,1.48853,8.2714,2.876,0.577435


In [43]:
out=np.array(outt)
out=np.squeeze(out)
evaluate(y_test.iloc[:,0], out)

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,13.9416,0.956156,6.80795,12.8928


## Applying RBFN to predict the value of all products for the next month

In [44]:
#Products= the products' names 
#nc=number of centers
#n_in= how much history
def tspredict1(Products,data,n_in,nc):
   dt=series_to_supervised(data,n_in,1)
   y=pd.DataFrame( dt.loc[:,'var1(t)':])
   X=dt.iloc[:,:(n_in*len(Products))]
   c=centers(X,nc)
   r=radial(X,c)
   inputs= pd.DataFrame(RBF(c,r,X))
   inputs__arr=np.array(inputs)
   y__arr=np.array(y)
   prediction=[]
   performance=pd.DataFrame()
   
   #m=MLPRegressor(activation='identity',alpha=0.05,learning_rate ='adaptive',verbose=False)
 
   for i in range(len(Products)):
        #mod=m.fit(inputs,y.iloc[:,i])
        model.fit(inputs__arr,y__arr[:,i], epochs=100, batch_size=20,verbose=1)
        pred=model.predict(inputs__arr)
        pred=np.squeeze(pred)
        ev=evaluate(y.iloc[:,i],pred)
        prediction.append(pred[len(pred)-1])
        performance=performance.append(ev)
   prediction=pd.DataFrame(prediction)
   return(prediction,performance)

In [45]:
pred,Performance=tspredict1(Products,dt,10,80)

Epoch 1/100
128/128 [==============================] - 0s - loss: 23.4778     
Epoch 2/100
128/128 [==============================] - 0s - loss: 15.9483     
Epoch 3/100
128/128 [==============================] - 0s - loss: 13.2641     
Epoch 4/100
128/128 [==============================] - 0s - loss: 12.1657     
Epoch 5/100
128/128 [==============================] - 0s - loss: 11.5801    
Epoch 6/100
128/128 [==============================] - 0s - loss: 11.5680     
Epoch 7/100
128/128 [==============================] - 0s - loss: 11.3110     
Epoch 8/100
128/128 [==============================] - 0s - loss: 11.2445     
Epoch 9/100
128/128 [==============================] - 0s - loss: 11.1871    
Epoch 10/100
128/128 [==============================] - 0s - loss: 11.0702     
Epoch 11/100
128/128 [==============================] - 0s - loss: 10.9983    
Epoch 12/100
128/128 [==============================] - 0s - loss: 10.9779    
Epoch 13/100
128/128 [==============================]

128/128 [==============================] - 0s - loss: 335.9216     
Epoch 3/100
128/128 [==============================] - 0s - loss: 274.2036     
Epoch 4/100
128/128 [==============================] - 0s - loss: 253.0722     
Epoch 5/100
128/128 [==============================] - 0s - loss: 245.2926     
Epoch 6/100
128/128 [==============================] - 0s - loss: 240.3020     
Epoch 7/100
128/128 [==============================] - 0s - loss: 236.0410     
Epoch 8/100
128/128 [==============================] - 0s - loss: 232.0250     
Epoch 9/100
128/128 [==============================] - 0s - loss: 230.3298     
Epoch 10/100
128/128 [==============================] - 0s - loss: 227.6352     
Epoch 11/100
128/128 [==============================] - 0s - loss: 228.3934     
Epoch 12/100
128/128 [==============================] - 0s - loss: 225.1340     
Epoch 13/100
128/128 [==============================] - 0s - loss: 223.5728     
Epoch 14/100
128/128 [==========================

128/128 [==============================] - 0s - loss: 164.9725     
Epoch 3/100
128/128 [==============================] - 0s - loss: 100.7310     
Epoch 4/100
128/128 [==============================] - 0s - loss: 73.0340     
Epoch 5/100
128/128 [==============================] - 0s - loss: 62.6229     
Epoch 6/100
128/128 [==============================] - 0s - loss: 57.1743     
Epoch 7/100
128/128 [==============================] - 0s - loss: 55.2132     
Epoch 8/100
128/128 [==============================] - 0s - loss: 53.1183     
Epoch 9/100
128/128 [==============================] - 0s - loss: 52.1095     
Epoch 10/100
128/128 [==============================] - 0s - loss: 51.4447     
Epoch 11/100
128/128 [==============================] - 0s - loss: 50.2039     
Epoch 12/100
128/128 [==============================] - 0s - loss: 49.3754     
Epoch 13/100
128/128 [==============================] - 0s - loss: 48.4690     
Epoch 14/100
128/128 [==============================] - 0s

128/128 [==============================] - 0s - loss: 15.6320     
Epoch 5/100
128/128 [==============================] - 0s - loss: 15.4286    
Epoch 6/100
128/128 [==============================] - 0s - loss: 15.1465     
Epoch 7/100
128/128 [==============================] - 0s - loss: 15.0348     
Epoch 8/100
128/128 [==============================] - 0s - loss: 14.8896    
Epoch 9/100
128/128 [==============================] - 0s - loss: 14.7306     
Epoch 10/100
128/128 [==============================] - 0s - loss: 14.4756     
Epoch 11/100
128/128 [==============================] - 0s - loss: 14.4157     
Epoch 12/100
128/128 [==============================] - 0s - loss: 14.1711    
Epoch 13/100
128/128 [==============================] - 0s - loss: 14.0265     
Epoch 14/100
128/128 [==============================] - 0s - loss: 13.8356     
Epoch 15/100
128/128 [==============================] - 0s - loss: 13.6934     
Epoch 16/100
128/128 [==============================] - 0s - 

128/128 [==============================] - 0s - loss: 5.6341     
Epoch 8/100
128/128 [==============================] - 0s - loss: 5.5203     
Epoch 9/100
128/128 [==============================] - 0s - loss: 5.5025     
Epoch 10/100
128/128 [==============================] - 0s - loss: 5.4310     
Epoch 11/100
128/128 [==============================] - 0s - loss: 5.4076     
Epoch 12/100
128/128 [==============================] - 0s - loss: 5.3414     
Epoch 13/100
128/128 [==============================] - 0s - loss: 5.3159      
Epoch 14/100
128/128 [==============================] - 0s - loss: 5.2728     
Epoch 15/100
128/128 [==============================] - 0s - loss: 5.2216     
Epoch 16/100
128/128 [==============================] - 0s - loss: 5.1884     
Epoch 17/100
128/128 [==============================] - 0s - loss: 5.1532     
Epoch 18/100
128/128 [==============================] - 0s - loss: 5.1295     
Epoch 19/100
128/128 [==============================] - 0s - loss:

...

128/128 [==============================] - 0s - loss: 0.4662     
Epoch 46/100
128/128 [==============================] - 0s - loss: 0.4692     
Epoch 47/100
128/128 [==============================] - 0s - loss: 0.4620     
Epoch 48/100
128/128 [==============================] - 0s - loss: 0.4621     
Epoch 49/100
128/128 [==============================] - 0s - loss: 0.4640     
Epoch 50/100
128/128 [==============================] - 0s - loss: 0.4614     
Epoch 51/100
128/128 [==============================] - 0s - loss: 0.4621     
Epoch 52/100
128/128 [==============================] - 0s - loss: 0.4586     
Epoch 53/100
128/128 [==============================] - 0s - loss: 0.4575     
Epoch 54/100
128/128 [==============================] - 0s - loss: 0.4563     
Epoch 55/100
128/128 [==============================] - 0s - loss: 0.4570     
Epoch 56/100
128/128 [==============================] - 0s - loss: 0.4602     
Epoch 57/100
128/128 [==============================] - 0s - loss

128/128 [==============================] - 0s - loss: 1.3881     
Epoch 49/100
128/128 [==============================] - 0s - loss: 1.3986     
Epoch 50/100
128/128 [==============================] - 0s - loss: 1.3975     
Epoch 51/100
128/128 [==============================] - 0s - loss: 1.3971     
Epoch 52/100
128/128 [==============================] - 0s - loss: 1.3897     
Epoch 53/100
128/128 [==============================] - 0s - loss: 1.3996     
Epoch 54/100
128/128 [==============================] - 0s - loss: 1.3898     
Epoch 55/100
128/128 [==============================] - 0s - loss: 1.4119     
Epoch 56/100
128/128 [==============================] - 0s - loss: 1.4054     
Epoch 57/100
128/128 [==============================] - 0s - loss: 1.3880     
Epoch 58/100
128/128 [==============================] - 0s - loss: 1.3944     
Epoch 59/100
128/128 [==============================] - 0s - loss: 1.4051     
Epoch 60/100
128/128 [==============================] - 0s - loss

128/128 [==============================] - 0s - loss: 0.2184     
Epoch 52/100
128/128 [==============================] - 0s - loss: 0.2177     
Epoch 53/100
128/128 [==============================] - 0s - loss: 0.2152     
Epoch 54/100
128/128 [==============================] - 0s - loss: 0.2139     
Epoch 55/100
128/128 [==============================] - 0s - loss: 0.2154     
Epoch 56/100
128/128 [==============================] - 0s - loss: 0.2136     
Epoch 57/100
128/128 [==============================] - 0s - loss: 0.2135     
Epoch 58/100
128/128 [==============================] - 0s - loss: 0.2120     
Epoch 59/100
128/128 [==============================] - 0s - loss: 0.2129     
Epoch 60/100
128/128 [==============================] - 0s - loss: 0.2117     
Epoch 61/100
128/128 [==============================] - 0s - loss: 0.2112     
Epoch 62/100
128/128 [==============================] - 0s - loss: 0.2105     
Epoch 63/100
128/128 [==============================] - 0s - loss

128/128 [==============================] - 0s - loss: 0.2126     
Epoch 55/100
128/128 [==============================] - 0s - loss: 0.2137     
Epoch 56/100
128/128 [==============================] - 0s - loss: 0.2158     
Epoch 57/100
128/128 [==============================] - 0s - loss: 0.2149     
Epoch 58/100
128/128 [==============================] - 0s - loss: 0.2127     
Epoch 59/100
128/128 [==============================] - 0s - loss: 0.2124     
Epoch 60/100
128/128 [==============================] - 0s - loss: 0.2142     
Epoch 61/100
128/128 [==============================] - 0s - loss: 0.2113     
Epoch 62/100
128/128 [==============================] - 0s - loss: 0.2135     
Epoch 63/100
128/128 [==============================] - 0s - loss: 0.2146     
Epoch 64/100
128/128 [==============================] - 0s - loss: 0.2134     
Epoch 65/100
128/128 [==============================] - 0s - loss: 0.2124     
Epoch 66/100
128/128 [==============================] - ETA: 0s -

128/128 [==============================] - 0s - loss: 1.3142     
Epoch 58/100
128/128 [==============================] - 0s - loss: 1.3138     
Epoch 59/100
128/128 [==============================] - 0s - loss: 1.3083     
Epoch 60/100
128/128 [==============================] - 0s - loss: 1.3228     
Epoch 61/100
128/128 [==============================] - 0s - loss: 1.3038     
Epoch 62/100
128/128 [==============================] - 0s - loss: 1.3070     
Epoch 63/100
128/128 [==============================] - 0s - loss: 1.2974     
Epoch 64/100
128/128 [==============================] - 0s - loss: 1.2910     
Epoch 65/100
128/128 [==============================] - 0s - loss: 1.2939     
Epoch 66/100
128/128 [==============================] - 0s - loss: 1.3050     
Epoch 67/100
128/128 [==============================] - 0s - loss: 1.2907     
Epoch 68/100
128/128 [==============================] - 0s - loss: 1.2924     
Epoch 69/100
128/128 [==============================] - 0s - loss

In [46]:
pred[pred<0]=0
Products=pd.DataFrame(Products)
Prediction=pd.concat([Products,pred],axis=1)

In [47]:
#one-month prediction for all products
Prediction

,ID,0
0,9.32E+15,0.581334
1,9.33E+15,26.487076
2,9243015,0.000000
3,9273011,0.000000
4,9273021,0.000000
5,9273031,0.937385
6,9273051,1.434957
7,9273052,1.526770
8,9273121,4.705204
9,9283021,1.854594


In [48]:
#Performance for each product prediction
Performance

,0,1,2,3
0,MAPE,MAD,MSD,SMAPE
1,20.0145,1.0834,6.47869,17.081
0,MAPE,MAD,MSD,SMAPE
1,54.2127,6.2911,137.368,24.7661
0,MAPE,MAD,MSD,SMAPE
1,58.3404,1.09115,7.35707,36.8651
0,MAPE,MAD,MSD,SMAPE
1,15.8286,0.201076,0.973233,9.19472
0,MAPE,MAD,MSD,SMAPE
1,0.80512,0.0161024,0.0331888,1.5625
